In [1]:
import mediapipe as mp
import cv2
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

Saving the Video

In [72]:
import time
cap = cv2.VideoCapture(0)
time.sleep(5)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc('P','I','M','1')
videoWriter = cv2.VideoWriter('lean.avi',fourcc,fps,(int(width),int(height)))

while cap.isOpened():
    ret,frame = cap.read()

    try:
        cv2.imshow('Press',frame)
        videoWriter.write(frame)
    except Exception as e:
        print(e)
        break
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

Capturing Landmarks and Saving it to CSV

In [43]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt

In [73]:
landmarks = ['class']  # Class is Y - target value
for val in range(1,33+1):
    landmarks+=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [74]:
with open('lean.csv',mode='w',newline='') as f:
    csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [75]:
# Function for exporting the landmarks to train
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        keypoints = np.insert(keypoints,0,action)

        with open('lean.csv',mode='a',newline='') as f: # a stands for append
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)
        pass

In [76]:
cap = cv2.VideoCapture('lean.avi')
# initiate the holistic model
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,image = cap.read()
        # Recolor the feed
        if image is None: break
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = pose.process(image)

        # Recoloring the image back for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS
                                  , mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        k = cv2.waitKey(1)
        if k==97: # a key 
            export_landmark(results,0) # 0 -> left 
        if k==115:# s key
            export_landmark(results,1) # 1 -> Centre
        if k==100:# d key
            export_landmark(results,2) # 2 -> Right

        cv2.imshow('Raw Cam Feed',image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Training the labelled Data on a Classification Model

In [77]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [78]:
df = pd.read_csv('lean.csv')
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,2.0,0.486863,0.327875,-0.488337,0.993785,0.487840,0.300638,-0.500518,0.988047,0.492339,...,0.536805,0.069007,0.676731,1.269443,0.228841,0.049455,0.575201,1.289941,0.357325,0.044117
1,2.0,0.487478,0.327466,-0.563560,0.994589,0.489192,0.300300,-0.569419,0.989206,0.494039,...,0.547285,0.076919,0.675951,1.265068,0.247452,0.052949,0.571935,1.288150,0.346206,0.048501
2,2.0,0.499366,0.323169,-0.628696,0.995261,0.499374,0.297625,-0.619610,0.990490,0.505360,...,0.574524,0.078438,0.682325,1.261319,0.305015,0.056239,0.571067,1.283025,0.367230,0.050384
3,2.0,0.502554,0.320817,-0.653112,0.995909,0.503170,0.295762,-0.641930,0.991762,0.509686,...,0.583987,0.078055,0.679628,1.256005,0.323335,0.058067,0.570486,1.276793,0.379827,0.050137
4,2.0,0.509379,0.319161,-0.656751,0.996615,0.510683,0.294830,-0.643753,0.993135,0.517237,...,0.586458,0.078207,0.681441,1.252536,0.345766,0.062272,0.568646,1.270885,0.392312,0.050663


In [79]:
X = df.drop('class',axis=1) # features
Y = df['class'] # label

In [80]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=1234)

In [81]:
# Training and Pipelining the Model
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

In [82]:
# Pipelines
pipelines = {
    'rf':make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [83]:
fit_models = {}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,Y_train)
    fit_models[algo] = model 

In [84]:
fit_models

{'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [85]:
fit_models['rf'].predict(X_test)

array([2., 0., 1., 1., 1., 1., 2., 1., 2., 2., 0., 1., 1., 2., 1., 2., 1.,
       2., 1., 0., 1., 1., 0., 2., 1., 0., 0., 2., 1., 1., 0., 0., 0., 0.,
       0., 2., 0., 2., 1., 2., 1., 2., 1., 2., 1., 1., 1., 2., 1., 1., 0.,
       2., 1., 1., 1., 2., 2., 0., 2., 2., 0., 1., 0., 1., 1., 2., 1., 2.,
       1., 0., 0., 0., 0.])

Evaluate and Serialize the Model


In [86]:
from sklearn.metrics import accuracy_score,precision_score,recall_score
import pickle

In [87]:
for algo,model in fit_models.items():
    Y_pred = model.predict(X_test)
    print(algo,accuracy_score(Y_test.values,Y_pred),
          precision_score(Y_test.values,Y_pred,average='weighted'),
          recall_score(Y_test.values,Y_pred,average='weighted'))

rf 0.958904109589041 0.9588418430884185 0.958904109589041
gb 0.958904109589041 0.9607721046077211 0.958904109589041


In [88]:
Y_pred = fit_models['gb'].predict(X_test)
Y_pred

array([2., 0., 1., 1., 1., 1., 2., 1., 2., 2., 0., 1., 1., 2., 1., 2., 1.,
       2., 1., 0., 1., 1., 0., 2., 1., 0., 0., 2., 1., 1., 0., 0., 0., 0.,
       0., 2., 0., 2., 1., 2., 1., 2., 1., 2., 0., 0., 1., 2., 1., 1., 0.,
       2., 1., 1., 1., 2., 2., 0., 2., 2., 0., 1., 0., 1., 1., 2., 1., 2.,
       1., 0., 0., 0., 0.])

In [89]:
with open('lean.pkl','wb') as f:
    pickle.dump(fit_models['gb'],f)

Making Detections with the Model

In [90]:
with open('lean.pkl','rb') as f:
    model = pickle.load(f)

In [91]:
cap = cv2.VideoCapture(0)
current_Stage = ''
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = pose.process(image)

        # Recolor it back
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # Draw Landmarks
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS
                                  , mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        try:
            row = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row],columns=landmarks[1:])
            bodylang_prob = model.predict_proba(X)[0]
            bodylang_class = model.predict(X)[0]
            
            if bodylang_class==0.0 and bodylang_prob[bodylang_prob.argmax()] > 0.7:
                current_Stage = "Left"
            elif bodylang_class==1.0 and bodylang_prob[bodylang_prob.argmax()] > 0.7:
                current_Stage = "Centre"
            elif bodylang_class==2.0 and bodylang_prob[bodylang_prob.argmax()] > 0.7:
                current_Stage = "Right"

            # Get status box
            cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1)

            # Display Class
            cv2.putText(image,'Class',
                        (95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,current_Stage,
                            (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            # if bodylang_class==0.0:
            #     cv2.putText(image,"LEFT",
            #                 (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            # elif bodylang_class==1.0:
            #     cv2.putText(image,"UP",
            #                 (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            # Display Probability
            cv2.putText(image,'Prob',
                        (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(round(bodylang_prob[np.argmax(bodylang_prob)],2)),
                        (10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            cv2.imshow('Deadlifts Lean',image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        except Exception as e:
            print(e)
            pass   
cap.release()
cv2.destroyAllWindows()

'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute 'landmark'
'NoneType' object has no attribute